## 1. Importación de Librerías

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import joblib
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, f1_score, roc_auc_score, confusion_matrix, accuracy_score
from sklearn.metrics import precision_recall_curve, auc
from sklearn.preprocessing import label_binarize

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.nn.functional as F

from tqdm import tqdm

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
df_device = pd.DataFrame({
    'Componente': ['Device', 'CUDA Disponible', 'Nombre GPU'],
    'Valor': [
        str(device),
        torch.cuda.is_available(),
        torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'
    ]
})

df_device

## 2. Configuración de Rutas

In [ ]:
ruta_base = Path(r'e:\06. Sexto Ciclo\01. Machine Learning\07. Workspace\16S03. Proyecto 03\P3-EcoSort')
ruta_features = ruta_base / 'result' / 'features'
ruta_modelos = ruta_base / 'result' / 'models'
ruta_figuras = ruta_base / 'result' / 'figures'

ruta_modelos.mkdir(parents=True, exist_ok=True)
ruta_figuras.mkdir(parents=True, exist_ok=True)

clases = ['general', 'paper', 'plastic']

## 3. Carga de Características

In [ ]:
X_train_img = np.load(ruta_features / 'X_train_imagenes.npy')
y_train = np.load(ruta_features / 'y_train.npy')
X_val_img = np.load(ruta_features / 'X_val_imagenes.npy')
y_val = np.load(ruta_features / 'y_val.npy')

X_train_pca = np.load(ruta_features / 'features_train_pca.npy')
X_val_pca = np.load(ruta_features / 'features_val_pca.npy')

df_carga = pd.DataFrame({
    'Dataset': ['Train Imágenes', 'Val Imágenes', 'Train PCA', 'Val PCA'],
    'Shape': [X_train_img.shape, X_val_img.shape, X_train_pca.shape, X_val_pca.shape],
    'Etiquetas': [len(y_train), len(y_val), len(y_train), len(y_val)]
})

df_carga

## 4. MODELO 1: Logistic Regression

### 4.1 Optimización de Hiperparámetros

In [ ]:
param_grid_lr = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga'],
    'max_iter': [1000, 2000]
}

modelo_lr_base = LogisticRegression(random_state=42)

grid_search_lr = GridSearchCV(
    modelo_lr_base, 
    param_grid_lr, 
    cv=5, 
    scoring='f1_macro', 
    n_jobs=-1,
    verbose=1
)

grid_search_lr.fit(X_train_pca, y_train)

mejores_params_lr = grid_search_lr.best_params_
mejor_score_lr = grid_search_lr.best_score_

In [ ]:
df_lr_params = pd.DataFrame({
    'Parámetro': list(mejores_params_lr.keys()) + ['F1 Macro CV'],
    'Valor': list(mejores_params_lr.values()) + [mejor_score_lr]
})

df_lr_params

### 4.2 Entrenamiento Final y Evaluación

In [ ]:
modelo_lr_final = grid_search_lr.best_estimator_

y_pred_train_lr = modelo_lr_final.predict(X_train_pca)
y_pred_val_lr = modelo_lr_final.predict(X_val_pca)

f1_macro_train_lr = f1_score(y_train, y_pred_train_lr, average='macro')
f1_micro_train_lr = f1_score(y_train, y_pred_train_lr, average='micro')
f1_macro_val_lr = f1_score(y_val, y_pred_val_lr, average='macro')
f1_micro_val_lr = f1_score(y_val, y_pred_val_lr, average='micro')

y_train_bin = label_binarize(y_train, classes=[0, 1, 2])
y_val_bin = label_binarize(y_val, classes=[0, 1, 2])
y_pred_proba_lr = modelo_lr_final.predict_proba(X_val_pca)

precision_lr = {}
recall_lr = {}
auc_pr_lr = {}

for i in range(3):
    precision_lr[i], recall_lr[i], _ = precision_recall_curve(y_val_bin[:, i], y_pred_proba_lr[:, i])
    auc_pr_lr[i] = auc(recall_lr[i], precision_lr[i])

auc_pr_macro_lr = np.mean(list(auc_pr_lr.values()))

df_lr_metricas = pd.DataFrame({
    'Métrica': ['F1 Macro Train', 'F1 Micro Train', 'F1 Macro Val', 'F1 Micro Val', 'AUC-PR Macro Val'],
    'Valor': [f1_macro_train_lr, f1_micro_train_lr, f1_macro_val_lr, f1_micro_val_lr, auc_pr_macro_lr]
})

df_lr_metricas['Valor'] = df_lr_metricas['Valor'].round(4)
df_lr_metricas

In [ ]:
cm_lr = confusion_matrix(y_val, y_pred_val_lr)

fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(cm_lr, annot=True, fmt='d', cmap='Blues', xticklabels=clases, yticklabels=clases, 
           cbar_kws={'label': 'Cantidad'}, linewidths=2, linecolor='black', ax=ax)
ax.set_xlabel('Predicción', fontsize=12, fontweight='bold')
ax.set_ylabel('Valor Real', fontsize=12, fontweight='bold')
ax.set_title(f'Matriz de Confusión - Logistic Regression\nF1 Macro: {f1_macro_val_lr:.4f}', 
            fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig(ruta_figuras / '03_train_01_lr_confusion.svg', format='svg', bbox_inches='tight')
plt.show()

### 4.3 Guardar Modelo

In [ ]:
joblib.dump(modelo_lr_final, ruta_modelos / 'logistic_regression.pkl')
joblib.dump(mejores_params_lr, ruta_modelos / 'logistic_regression_params.pkl')

metricas_lr = {
    'f1_macro_train': f1_macro_train_lr,
    'f1_micro_train': f1_micro_train_lr,
    'f1_macro_val': f1_macro_val_lr,
    'f1_micro_val': f1_micro_val_lr,
    'auc_pr_macro_val': auc_pr_macro_lr
}
joblib.dump(metricas_lr, ruta_modelos / 'logistic_regression_metricas.pkl')

## 5. MODELO 2: SVM

### 5.1 Optimización de Hiperparámetros

In [ ]:
param_distributions_svm = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.001, 0.01, 0.1],
    'kernel': ['rbf', 'poly', 'sigmoid']
}

modelo_svm_base = SVC(random_state=42, probability=True)

random_search_svm = RandomizedSearchCV(
    modelo_svm_base,
    param_distributions_svm,
    n_iter=30,
    cv=5,
    scoring='f1_macro',
    n_jobs=-1,
    random_state=42,
    verbose=1
)

random_search_svm.fit(X_train_pca, y_train)

mejores_params_svm = random_search_svm.best_params_
mejor_score_svm = random_search_svm.best_score_

In [ ]:
df_svm_params = pd.DataFrame({
    'Parámetro': list(mejores_params_svm.keys()) + ['F1 Macro CV'],
    'Valor': [str(v) for v in mejores_params_svm.values()] + [mejor_score_svm]
})

df_svm_params

### 5.2 Entrenamiento Final y Evaluación

In [ ]:
modelo_svm_final = random_search_svm.best_estimator_

y_pred_train_svm = modelo_svm_final.predict(X_train_pca)
y_pred_val_svm = modelo_svm_final.predict(X_val_pca)

f1_macro_train_svm = f1_score(y_train, y_pred_train_svm, average='macro')
f1_micro_train_svm = f1_score(y_train, y_pred_train_svm, average='micro')
f1_macro_val_svm = f1_score(y_val, y_pred_val_svm, average='macro')
f1_micro_val_svm = f1_score(y_val, y_pred_val_svm, average='micro')

y_pred_proba_svm = modelo_svm_final.predict_proba(X_val_pca)

precision_svm = {}
recall_svm = {}
auc_pr_svm = {}

for i in range(3):
    precision_svm[i], recall_svm[i], _ = precision_recall_curve(y_val_bin[:, i], y_pred_proba_svm[:, i])
    auc_pr_svm[i] = auc(recall_svm[i], precision_svm[i])

auc_pr_macro_svm = np.mean(list(auc_pr_svm.values()))

df_svm_metricas = pd.DataFrame({
    'Métrica': ['F1 Macro Train', 'F1 Micro Train', 'F1 Macro Val', 'F1 Micro Val', 'AUC-PR Macro Val'],
    'Valor': [f1_macro_train_svm, f1_micro_train_svm, f1_macro_val_svm, f1_micro_val_svm, auc_pr_macro_svm]
})

df_svm_metricas['Valor'] = df_svm_metricas['Valor'].round(4)
df_svm_metricas

In [ ]:
cm_svm = confusion_matrix(y_val, y_pred_val_svm)

fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(cm_svm, annot=True, fmt='d', cmap='Greens', xticklabels=clases, yticklabels=clases, 
           cbar_kws={'label': 'Cantidad'}, linewidths=2, linecolor='black', ax=ax)
ax.set_xlabel('Predicción', fontsize=12, fontweight='bold')
ax.set_ylabel('Valor Real', fontsize=12, fontweight='bold')
ax.set_title(f'Matriz de Confusión - SVM\nF1 Macro: {f1_macro_val_svm:.4f}', 
            fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig(ruta_figuras / '03_train_02_svm_confusion.svg', format='svg', bbox_inches='tight')
plt.show()

### 5.3 Guardar Modelo

In [ ]:
joblib.dump(modelo_svm_final, ruta_modelos / 'svm.pkl')
joblib.dump(mejores_params_svm, ruta_modelos / 'svm_params.pkl')

metricas_svm = {
    'f1_macro_train': f1_macro_train_svm,
    'f1_micro_train': f1_micro_train_svm,
    'f1_macro_val': f1_macro_val_svm,
    'f1_micro_val': f1_micro_val_svm,
    'auc_pr_macro_val': auc_pr_macro_svm
}
joblib.dump(metricas_svm, ruta_modelos / 'svm_metricas.pkl')

## 6. MODELO 3: CNN con PyTorch

### 6.1 Definición de Arquitectura CNN

In [ ]:
class CNN_Clasificador(nn.Module):
    def __init__(self, num_clases=3, dropout=0.5):
        super(CNN_Clasificador, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(2, 2)
        
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(128)
        self.pool2 = nn.MaxPool2d(2, 2)
        
        self.conv5 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(256)
        self.conv6 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.bn6 = nn.BatchNorm2d(256)
        self.pool3 = nn.MaxPool2d(2, 2)
        
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(256 * 16 * 16, 512)
        self.dropout1 = nn.Dropout(dropout)
        self.fc2 = nn.Linear(512, 128)
        self.dropout2 = nn.Dropout(dropout)
        self.fc3 = nn.Linear(128, num_clases)
    
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool1(x)
        
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool2(x)
        
        x = F.relu(self.bn5(self.conv5(x)))
        x = F.relu(self.bn6(self.conv6(x)))
        x = self.pool3(x)
        
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        
        return x

### 6.2 Preparación de Datos para PyTorch

In [ ]:
X_train_tensor = torch.FloatTensor(X_train_img).permute(0, 3, 1, 2)
y_train_tensor = torch.LongTensor(y_train)
X_val_tensor = torch.FloatTensor(X_val_img).permute(0, 3, 1, 2)
y_val_tensor = torch.LongTensor(y_val)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

### 6.3 Función de Entrenamiento

In [ ]:
def entrenar_cnn(modelo, train_loader, val_loader, criterion, optimizer, num_epochs=50):
    historial_train_loss = []
    historial_val_loss = []
    historial_train_acc = []
    historial_val_acc = []
    
    mejor_val_loss = float('inf')
    mejor_modelo = None
    
    for epoch in range(num_epochs):
        modelo.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} - Train'):
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = modelo(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
        
        train_loss = train_loss / train_total
        train_acc = train_correct / train_total
        
        modelo.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                
                outputs = modelo(inputs)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        
        val_loss = val_loss / val_total
        val_acc = val_correct / val_total
        
        historial_train_loss.append(train_loss)
        historial_val_loss.append(val_loss)
        historial_train_acc.append(train_acc)
        historial_val_acc.append(val_acc)
        
        tqdm.write(f'Epoch {epoch+1}: Train Loss={train_loss:.4f}, Val Loss={val_loss:.4f}, '
                  f'Train Acc={train_acc:.4f}, Val Acc={val_acc:.4f}')
        
        if val_loss < mejor_val_loss:
            mejor_val_loss = val_loss
            mejor_modelo = modelo.state_dict().copy()
    
    modelo.load_state_dict(mejor_modelo)
    
    return modelo, historial_train_loss, historial_val_loss, historial_train_acc, historial_val_acc

### 6.4 Entrenamiento del Modelo CNN

In [ ]:
modelo_cnn = CNN_Clasificador(num_clases=3, dropout=0.5).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(modelo_cnn.parameters(), lr=0.001, weight_decay=1e-4)

modelo_cnn, train_loss_cnn, val_loss_cnn, train_acc_cnn, val_acc_cnn = entrenar_cnn(
    modelo_cnn, train_loader, val_loader, criterion, optimizer, num_epochs=50
)

### 6.5 Visualización de Curvas de Pérdida

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

epochs = range(1, len(train_loss_cnn) + 1)

axes[0].plot(epochs, train_loss_cnn, 'b-', linewidth=2, label='Train Loss', marker='o', markersize=4)
axes[0].plot(epochs, val_loss_cnn, 'r-', linewidth=2, label='Validation Loss', marker='s', markersize=4)
axes[0].set_xlabel('Epoch', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Loss', fontsize=12, fontweight='bold')
axes[0].set_title('Curvas de Pérdida - CNN', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

axes[1].plot(epochs, train_acc_cnn, 'b-', linewidth=2, label='Train Accuracy', marker='o', markersize=4)
axes[1].plot(epochs, val_acc_cnn, 'r-', linewidth=2, label='Validation Accuracy', marker='s', markersize=4)
axes[1].set_xlabel('Epoch', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Accuracy', fontsize=12, fontweight='bold')
axes[1].set_title('Curvas de Accuracy - CNN', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(ruta_figuras / '03_train_03_cnn_curvas.svg', format='svg', bbox_inches='tight')
plt.show()

### 6.6 Evaluación del Modelo CNN

In [ ]:
def evaluar_modelo_pytorch(modelo, data_loader):
    modelo.eval()
    todas_predicciones = []
    todas_etiquetas = []
    todas_probabilidades = []
    
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs = inputs.to(device)
            outputs = modelo(inputs)
            probabilidades = F.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)
            
            todas_predicciones.extend(predicted.cpu().numpy())
            todas_etiquetas.extend(labels.numpy())
            todas_probabilidades.extend(probabilidades.cpu().numpy())
    
    return np.array(todas_predicciones), np.array(todas_etiquetas), np.array(todas_probabilidades)

y_pred_train_cnn, y_true_train_cnn, y_proba_train_cnn = evaluar_modelo_pytorch(modelo_cnn, train_loader)
y_pred_val_cnn, y_true_val_cnn, y_proba_val_cnn = evaluar_modelo_pytorch(modelo_cnn, val_loader)

f1_macro_train_cnn = f1_score(y_true_train_cnn, y_pred_train_cnn, average='macro')
f1_micro_train_cnn = f1_score(y_true_train_cnn, y_pred_train_cnn, average='micro')
f1_macro_val_cnn = f1_score(y_true_val_cnn, y_pred_val_cnn, average='macro')
f1_micro_val_cnn = f1_score(y_true_val_cnn, y_pred_val_cnn, average='micro')

y_true_val_bin_cnn = label_binarize(y_true_val_cnn, classes=[0, 1, 2])

precision_cnn = {}
recall_cnn = {}
auc_pr_cnn = {}

for i in range(3):
    precision_cnn[i], recall_cnn[i], _ = precision_recall_curve(y_true_val_bin_cnn[:, i], y_proba_val_cnn[:, i])
    auc_pr_cnn[i] = auc(recall_cnn[i], precision_cnn[i])

auc_pr_macro_cnn = np.mean(list(auc_pr_cnn.values()))

df_cnn_metricas = pd.DataFrame({
    'Métrica': ['F1 Macro Train', 'F1 Micro Train', 'F1 Macro Val', 'F1 Micro Val', 'AUC-PR Macro Val'],
    'Valor': [f1_macro_train_cnn, f1_micro_train_cnn, f1_macro_val_cnn, f1_micro_val_cnn, auc_pr_macro_cnn]
})

df_cnn_metricas['Valor'] = df_cnn_metricas['Valor'].round(4)
df_cnn_metricas

In [ ]:
cm_cnn = confusion_matrix(y_true_val_cnn, y_pred_val_cnn)

fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(cm_cnn, annot=True, fmt='d', cmap='Reds', xticklabels=clases, yticklabels=clases, 
           cbar_kws={'label': 'Cantidad'}, linewidths=2, linecolor='black', ax=ax)
ax.set_xlabel('Predicción', fontsize=12, fontweight='bold')
ax.set_ylabel('Valor Real', fontsize=12, fontweight='bold')
ax.set_title(f'Matriz de Confusión - CNN\nF1 Macro: {f1_macro_val_cnn:.4f}', 
            fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig(ruta_figuras / '03_train_04_cnn_confusion.svg', format='svg', bbox_inches='tight')
plt.show()

### 6.7 Guardar Modelo CNN

In [ ]:
torch.save(modelo_cnn.state_dict(), ruta_modelos / 'cnn_model.pth')

historial_cnn = {
    'train_loss': train_loss_cnn,
    'val_loss': val_loss_cnn,
    'train_acc': train_acc_cnn,
    'val_acc': val_acc_cnn
}
joblib.dump(historial_cnn, ruta_modelos / 'cnn_historial.pkl')

metricas_cnn = {
    'f1_macro_train': f1_macro_train_cnn,
    'f1_micro_train': f1_micro_train_cnn,
    'f1_macro_val': f1_macro_val_cnn,
    'f1_micro_val': f1_micro_val_cnn,
    'auc_pr_macro_val': auc_pr_macro_cnn
}
joblib.dump(metricas_cnn, ruta_modelos / 'cnn_metricas.pkl')

## 7. Resumen de Entrenamiento

In [ ]:
df_resumen = pd.DataFrame({
    'Modelo': ['Logistic Regression', 'SVM', 'CNN'],
    'F1 Macro Train': [f1_macro_train_lr, f1_macro_train_svm, f1_macro_train_cnn],
    'F1 Macro Val': [f1_macro_val_lr, f1_macro_val_svm, f1_macro_val_cnn],
    'F1 Micro Val': [f1_micro_val_lr, f1_micro_val_svm, f1_micro_val_cnn],
    'AUC-PR Val': [auc_pr_macro_lr, auc_pr_macro_svm, auc_pr_macro_cnn]
})

df_resumen = df_resumen.round(4)
df_resumen

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

modelos = df_resumen['Modelo']
colores_modelos = ['#3498db', '#2ecc71', '#e74c3c']

axes[0].bar(modelos, df_resumen['F1 Macro Val'], alpha=0.8, color=colores_modelos, edgecolor='black')
axes[0].set_ylabel('F1 Macro Score', fontsize=12, fontweight='bold')
axes[0].set_title('F1 Macro - Validación', fontsize=14, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)
for i, v in enumerate(df_resumen['F1 Macro Val']):
    axes[0].text(i, v + 0.01, f'{v:.4f}', ha='center', fontsize=11, fontweight='bold')

axes[1].bar(modelos, df_resumen['F1 Micro Val'], alpha=0.8, color=colores_modelos, edgecolor='black')
axes[1].set_ylabel('F1 Micro Score', fontsize=12, fontweight='bold')
axes[1].set_title('F1 Micro - Validación', fontsize=14, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)
for i, v in enumerate(df_resumen['F1 Micro Val']):
    axes[1].text(i, v + 0.01, f'{v:.4f}', ha='center', fontsize=11, fontweight='bold')

axes[2].bar(modelos, df_resumen['AUC-PR Val'], alpha=0.8, color=colores_modelos, edgecolor='black')
axes[2].set_ylabel('AUC-PR Score', fontsize=12, fontweight='bold')
axes[2].set_title('AUC-PR Macro - Validación', fontsize=14, fontweight='bold')
axes[2].grid(axis='y', alpha=0.3)
for i, v in enumerate(df_resumen['AUC-PR Val']):
    axes[2].text(i, v + 0.01, f'{v:.4f}', ha='center', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig(ruta_figuras / '03_train_05_comparacion_modelos.svg', format='svg', bbox_inches='tight')
plt.show()

### Conclusiones:

**1. Modelos Entrenados:**
- Logistic Regression: Modelo base simple con optimización de hiperparámetros
- SVM: Modelo avanzado con kernel optimizado mediante Random Search
- CNN: Red neuronal convolucional profunda entrenada con PyTorch/CUDA

**2. Optimización:**
- Cada modelo fue optimizado exhaustivamente variando todos sus hiperparámetros
- Cross-validation utilizado para validar hiperparámetros
- Métricas orientadas al problema: F1 Macro como objetivo principal

**3. Curvas de Pérdida:**
- CNN muestra convergencia clara en las curvas train vs validation
- Todas las curvas guardadas para análisis posterior

**4. Artefactos Guardados:**
- Modelos entrenados guardados en result/models/
- Hiperparámetros óptimos y métricas almacenadas
- Curvas de pérdida guardadas para evaluación posterior